In [1]:
import evaluate
import json
from tqdm import tqdm

In [2]:
with open("./Outputs/StructLm-7B/StructLm-7B.json", 'r') as f:
    data = json.load(f)

In [16]:
def evaluating(org, pred, metric_, value_):
    dict_metric = {}
    metric = evaluate.load(metric_)
    metric_per_review = 0
    idx = 0
    for i in tqdm(range(len(org))):
        review_org = org[i]
        review_pred = pred[i]
        try:
            for key, val in review_org.items():
                if key not in dict_metric:
                    dict_metric[key] = [0, 0] 
                val_pred = [str(review_pred[key])]
                if metric_ == 'bleu':        
                    score = metric.compute(references=[str(val)], predictions=val_pred, max_order=2, smooth=True)[value_]
                else:
                    score = metric.compute(references=[str(val)], predictions=val_pred)[value_]
                dict_metric[key][0] += score
                dict_metric[key][1] += 1
                idx += 1
        except:
            idx += 1
    sum = 0
    rest = 0
    for key in dict_metric.keys():
        if dict_metric[key][1] != 0:
            dict_metric[key][0] = round((dict_metric[key][0]/dict_metric[key][1])*100, 2)
        else:
            rest += 1
        sum += dict_metric[key][0]
    if len(dict_metric.keys()) - rest == 0:

        dict_metric["Total"] = [0]
    dict_metric["Total"] = [round(sum/(len(dict_metric.keys()) - rest), 2), len(dict_metric.keys()) - rest]
    return dict_metric

In [4]:
Original = data['Original']
Pred = data['Prediction']
bleu = evaluating(Original, Pred, 'bleu', 'bleu')
rougeL = evaluating(Original, Pred, "rouge", "rougeL")
rouge1 = evaluating(Original, Pred, "rouge", "rouge1")
rouge2 = evaluating(Original, Pred, "rouge", "rouge2")
rougeLsum = evaluating(Original, Pred, "rouge", "rougeLsum")
meteor = evaluating(Original, Pred, "meteor", "meteor")

100%|██████████| 4978/4978 [08:54<00:00,  9.32it/s]


[nltk_data] Downloading package wordnet to /home/luis/nltk_data...
[nltk_data] Downloading package punkt_tab to /home/luis/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /home/luis/nltk_data...
100%|██████████| 4978/4978 [00:31<00:00, 156.66it/s]


In [10]:
metrics = {
    "bleu" : bleu,
    "rouge1" : rouge1,
    "rouge2" : rouge2,
    "rougeL" : rougeL,
    "rougeLsum" : rougeLsum,
    "meteor" : meteor
}

In [15]:
with open("./metrics/StructLm-7B/StructLm-7B.json", 'w+') as f:
    json.dump(metrics, f, indent=4 ,ensure_ascii=False)